# **Computational Theory Assessment**

**Scope**  
This notebook presents solutions to all required tasks in the Computational Theory module, organized into distinct sections. Each task is supported by explanatory Markdown cells and corresponding Python code cells that implement and test the solutions. This structure ensures clarity and demonstrates a step-by-step approach to each assignment requirement.

**Objectives**  
1. Showcase implementation details for binary manipulations, hashing, prime calculations, Turing machines, and more.  
2. Provide thorough test coverage with `unittest` to confirm correctness and reliability of the functions.  
3. Document each task’s purpose, method, and cryptographic or theoretical context, aligning with the assignment guidelines.

All content is arranged so that a reader can navigate the tasks in sequence, understanding both the conceptual underpinnings and practical code execution.


## **Imports and Libraries**

The set of libraries used throughout this notebook:

- **unittest** – Provides a framework for structured testing of each function.  
- **tempfile** and **os** – Facilitate file creation and management, particularly for tasks involving SHA-256 padding demonstrations.  
- **math** – Supplies mathematical functions like `sqrt` and `floor`, especially for prime-related or fractional bit extraction tasks.  
- **pandas** – Assists with tabular data representation in certain demonstrations.  
- **hashlib** – Used for SHA-256 hashing functionality in proof-of-work and padding tasks.  
- **itertools** – Generates permutations and combinations, vital for tasks like bubble sort comparisons.  
- **tabulate** – Enables readable table formatting for selected outputs.  


In [227]:
import unittest
import tempfile
import os
import math
import pandas as pd
import hashlib
import itertools
from tabulate import tabulate

## **Task 1 - Binary Representations**

**Purpose**

- This task implements four core bitwise operations: `rotl`, `rotr`, `ch`, and `maj`.

- These operations are widely used in cryptographic algorithms such as SHA-256 to ensure strong diffusion and non-linearity.

- Bitwise logic ensures that small changes in input produce large, complex changes in output, which is essential for secure hash design.

- Each function is tested using `unittest` to validate correctness and ensure reliable behavior in all edge cases.

**Cryptographic Context**

- These functions mirror components in SHA-256:

  - `rotl` and `rotr` allow cyclic shifting of bits to prevent data loss during mixing.

  - `ch` (choose) and `maj` (majority) contribute to conditional and majority logic in the SHA-256 compression function.
  
- Together, they provide the low-level operations needed for constructing secure, high-entropy hash outputs.


### **Step 1: Implement Bitwise Left Rotation - `rotl(x, n=1)`**

- **Definition**
    - Performs a left circular rotation on a 32-bit unsigned integer.

    - Bits shifted off the left end reappear on the right.

    - A 32-bit mask ensures results stay within the valid integer range.

    - Left rotation is used in hash functions to mix bits without zero-padding.
    
    - Ensures full bit coverage and contributes to the avalanche effect in cryptographic diffusion.

In [112]:
MASK_32 = 0xFFFFFFFF  # 32-bit mask to prevent overflow

def rotl(x, n=1):
    n %= 32  # Ensures n is within 0-31
    return ((x << n) & MASK_32) | ((x & MASK_32) >> (32 - n))

#### **Test Class: `TestRotlOutput`**

- **Test Approach**

    - Applies `rotl` to a known test value with shifts of 0, 4, 8, and 31.

    - Uses both hex and binary output for readability.

- **Verification**

    - Each result is compared to a precomputed expected output.

- **Outcome**

    - All results match expected values, confirming correct bit rotation and masking logic.

In [174]:
def to_bin_str(val):
    return bin(val)[2:].zfill(32)

class TestRotlOutput(unittest.TestCase):
    def test_rotl_output(self):
        print("\nStarting validation for ROTL (rotate left) function...\n")

        test_val = 0x12345678
        test_cases = {
            0:  0x12345678,
            4:  0x23456781,
            8:  0x34567812,
            31: 0x091A2B3C
        }

        print(f"Original:   0x{test_val:X} ({to_bin_str(test_val)})\n")

        for n, expected in test_cases.items():
            result = rotl(test_val, n)
            print(f"rotl({n}):   0x{result:X} ({to_bin_str(result)})")
            self.assertEqual(result, expected)
            print(f"Test passed: rotl({n}) produces correct result\n")

        print("All ROTL tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestRotlOutput))


test_rotl_output (__main__.TestRotlOutput.test_rotl_output) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation for ROTL (rotate left) function...

Original:   0x12345678 (00010010001101000101011001111000)

rotl(0):   0x12345678 (00010010001101000101011001111000)
Test passed: rotl(0) produces correct result

rotl(4):   0x23456781 (00100011010001010110011110000001)
Test passed: rotl(4) produces correct result

rotl(8):   0x34567812 (00110100010101100111100000010010)
Test passed: rotl(8) produces correct result

rotl(31):   0x91A2B3C (00001001000110100010101100111100)
Test passed: rotl(31) produces correct result

All ROTL tests completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 2: Implement Bitwise Right Rotation - `rotr(x, n=1)`**

- **Definition**

    - Performs a right circular rotation on a 32-bit unsigned integer.

    - Bits shifted off the right end re-enter from the left.

    - The result is masked to 32 bits to match unsigned integer constraints.

    - Right rotations appear in cryptographic functions like σ0 and σ1 in SHA-256.

    - They provide complementary behavior to left rotations and support full-state mixing across rounds.

In [188]:
MASK_32 = 0xFFFFFFFF  # 32-bit mask to prevent overflow

def rotr(x, n=1):
    n %= 32  # Ensure n is within 0-31
    return ((x >> n) | (x << (32 - n))) & MASK_32

#### **Test Class: `TestRotrOutput`**

- **Test Approach**
    - Uses the same test value as `rotl` and applies right rotations by 1, 4, 8, and 31.

- **Verification**
    - Checks whether rotated outputs match expected binary patterns.

- **Outcome**
    - All outputs align with known reference results, validating the rotation logic.

In [189]:
def to_bin_str(val):
    return bin(val)[2:].zfill(32)

class TestRotrOutput(unittest.TestCase):
    def test_rotr_output(self):
        print("\nStarting validation for ROTR (rotate right) function...\n")

        test_val = 0x12345678
        test_cases = {
            1:  0x091A2B3C, 
            4:  0x81234567,
            8:  0x78123456,
            31: 0x2468ACF0
        }

        print(f"Original:   0x{test_val:X} ({to_bin_str(test_val)})\n")

        for n, expected in test_cases.items():
            result = rotr(test_val, n)
            print(f"rotr({n}):   0x{result:X} ({to_bin_str(result)})")
            self.assertEqual(result, expected)
            print(f"Test passed: rotr({n}) produces correct result\n")

        print("All ROTR tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestRotrOutput))

test_rotr_output (__main__.TestRotrOutput.test_rotr_output) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK



Starting validation for ROTR (rotate right) function...

Original:   0x12345678 (00010010001101000101011001111000)

rotr(1):   0x91A2B3C (00001001000110100010101100111100)
Test passed: rotr(1) produces correct result

rotr(4):   0x81234567 (10000001001000110100010101100111)
Test passed: rotr(4) produces correct result

rotr(8):   0x78123456 (01111000000100100011010001010110)
Test passed: rotr(8) produces correct result

rotr(31):   0x2468ACF0 (00100100011010001010110011110000)
Test passed: rotr(31) produces correct result

All ROTR tests completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

#### **Step 3: Implement `ch(x, y, z)` Function**

- **Definition**
    - For each bit position, chooses the bit from `y` if `x` has a 1, otherwise takes it from `z`.
    - Implements the SHA-256 `Ch` function: `(x & y) ^ (~x & z)`.
    - Introduces conditional logic based on the state of control bits.
    - This selectiveness adds non-linearity and makes output harder to predict or reverse.

In [116]:
# Chooses bits from y where x has bits set to 1 and from z where x has bits set to 0.
def ch(x, y, z):
    return (x & y) ^ (~x & z)


#### **Test Class: `TestChFunction`**

- **Test Approach**
    - Applies the function to controlled binary inputs that represent corner cases (e.g., all 0s, all 1s, mixed).

- **Verification**
    - Asserts output bits correctly reflect the behavior of `ch`.

- **Outcome**
    - Passes all logic-based edge cases, confirming the conditional bit selection is implemented properly.

In [176]:
class TestChFunction(unittest.TestCase):
    def test_ch_original_cases(self):
        print("\nStarting validation for ch(x, y, z) function...\n")

        # Test 1: x is different from both y and z
        x_val = 0b1010
        y_val = 0b1100
        z_val = 0b1111
        result = ch(x_val, y_val, z_val) & 0b1111
        print(f"ch(0b{x_val:04b}, 0b{y_val:04b}, 0b{z_val:04b}) = 0b{result:04b}")
        self.assertEqual(result, 0b1101)
        print("Test passed: x ≠ y and x ≠ z\n")

        # Test 2: x is equal to y and different from z
        x_val = 0b0000
        y_val = 0b0000
        z_val = 0b1111
        result = ch(x_val, y_val, z_val) & 0b1111
        print(f"ch(0b{x_val:04b}, 0b{y_val:04b}, 0b{z_val:04b}) = 0b{result:04b}")
        self.assertEqual(result, 0b1111)
        print("Test passed: x == y and x ≠ z\n")

        # Test 3: x is equal to z and different from y
        x_val = 0b1111
        y_val = 0b1100
        z_val = 0b1111
        result = ch(x_val, y_val, z_val) & 0b1111
        print(f"ch(0b{x_val:04b}, 0b{y_val:04b}, 0b{z_val:04b}) = 0b{result:04b}")
        self.assertEqual(result, 0b1100)
        print("Test passed: x == z and x ≠ y\n")

        print("All ch(x, y, z) tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestChFunction))


test_ch_original_cases (__main__.TestChFunction.test_ch_original_cases) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation for ch(x, y, z) function...

ch(0b1010, 0b1100, 0b1111) = 0b1101
Test passed: x ≠ y and x ≠ z

ch(0b0000, 0b0000, 0b1111) = 0b1111
Test passed: x == y and x ≠ z

ch(0b1111, 0b1100, 0b1111) = 0b1100
Test passed: x == z and x ≠ y

All ch(x, y, z) tests completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 4: Implement `maj(x, y, z)` Function**

- **Definition**

    - Returns a bit that is 1 if at least two of the three corresponding bits are 1.

    - Implements the SHA-256 `Maj` function: `(x & y) ^ (x & z) ^ (y & z)`.

In [118]:
# Implement a function that takes a majority vote of the bits in x, y, z.
# Each bit should be 1 if at least two of three inputs have 1 in that position.
def maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

#### **Test Class: `TestMajFunction`**

- **Test Approach**
    - Tests three combinations of inputs: all 1s, all 0s, and a mixed case.
    
    - Validates behavior when x has dominant or minority influence.

- **Verification**
    - Confirms that the result reflects the majority logic at every bit position.

- **Outcome**
    - All tests pass, demonstrating correct implementation of majority function logic.

In [ ]:
class TestMajFunction(unittest.TestCase):
    def test_maj_cases(self):
        print("\nStarting validation for maj(x, y, z) function...\n")

        # Test 1: x has a mixture of 1s and 0s
        x_val = 0b1010
        y_val = 0b1100
        z_val = 0b1111
        result = maj(x_val, y_val, z_val) & 0b1111
        print(f"maj(0b{x_val:04b}, 0b{y_val:04b}, 0b{z_val:04b}) = 0b{result:04b}")
        self.assertEqual(result, 0b1110)
        print("Test passed: x has mixed 1s and 0s\n")

        # Test 2: x has all 1s
        x_val = 0b1111
        y_val = 0b1100
        z_val = 0b1111
        result = maj(x_val, y_val, z_val) & 0b1111
        print(f"maj(0b{x_val:04b}, 0b{y_val:04b}, 0b{z_val:04b}) = 0b{result:04b}")
        self.assertEqual(result, 0b1111)
        print("Test passed: x has all 1s\n")

        # Test 3: x has all 0s
        x_val = 0b0000
        y_val = 0b1100
        z_val = 0b1111
        result = maj(x_val, y_val, z_val) & 0b1111
        print(f"maj(0b{x_val:04b}, 0b{y_val:04b}, 0b{z_val:04b}) = 0b{result:04b}")
        self.assertEqual(result, 0b1100)
        print("Test passed: x has all 0s\n")

        print("All maj(x, y, z) tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestMajFunction))


test_maj_cases (__main__.TestMajFunction.test_maj_cases) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation for maj(x, y, z) function...

maj(0b1010, 0b1100, 0b1111) = 0b1110
Test passed: x has mixed 1s and 0s

maj(0b1111, 0b1100, 0b1111) = 0b1111
Test passed: x has all 1s

maj(0b0000, 0b1100, 0b1111) = 0b1100
Test passed: x has all 0s

All maj(x, y, z) tests completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Conclusion: Bitwise Foundations for Cryptographic Operations**

- This task successfully implemented and validated core 32-bit bitwise operations used in cryptographic hash functions.

- The tests confirmed correct behavior in various shift and logic scenarios.

- These operations form the foundation for more complex transformations in algorithms like SHA-256, where diffusion and predictability resistance are critical.

## **Task 2 - Hash Functions**

**Purpose**  

- Task 2 introduces a classic string-hashing approach from Kernighan & Ritchie (K&R). 

- This demonstrates how a simple polynomial rolling hash formula leverages prime constants to minimize collisions. 

- Prime-based distribution is key for hash tables, although the resulting hash is not cryptographically secure.

- The assignment also includes an “expanded” variant to show each calculation step in full.

- The two functions are:

    1. **hash_function(s)** – a concise K&R style hash.  

    2. **hash_function_expanded(s)** – prints intermediate steps for teaching or debugging.

- Both are tested thoroughly to ensure correctness.

**Historical / Cryptographic Background**  

- **K&R Hash** 
    - Published in “The C Programming Language” by Kernighan & Ritchie, used widely in early C programs for symbol tables.  

- **Prime Constants**
    - Multiplying by **31**, then taking modulus **101**, helps spread out hash values by exploiting properties of prime arithmetic. 

- **Non-Cryptographic**
    - This design is not intended for security (like SHA-256 or HMAC), but it illustrates fundamental hashing principles used in compilers, language interpreters, and basic hash tables.


### **Step 1: Convert C Hash Function to Python**

- **Definition**  

    - Each character’s ASCII value (`ord(c)`) is added after multiplying the current hash by 31.
    
    - The final result is constrained to `[0..100]` by taking `% 101`.


In [120]:
# Converts a string to a hash value.
def hash_function(s):

    hashValue = 0
    # Hash value updated for each character in the string
    for char in s:
        # ord() gets ASCII value of the character
        hashValue = ord(char) + 31 * hashValue
    # Ensure the hash value is within 0-100
    return hashValue % 101

#### **Test Class**: `TestHashFunction`  

- **Test Approach**: 

  - A set of predefined strings (e.g., `"hello"`, `"world"`, `"python"`) is passed through `hash_function`.  

- **Verification**

  - Each result is verified against a known correct integer. 

- **Outcome**
    - This process confirms the function’s internal arithmetic matches the K&R formula, and that each character properly shifts the running sum by multiplying it by 31 and adding the new ASCII code.

In [121]:
class TestHashFunction(unittest.TestCase):
    def test_hash_strings(self):
        print("\nStarting validation for hash_function(s)...\n")

        test_cases = [
            ("hello",    17),
            ("world",    34),
            ("python",   91),
            ("hash",     15),
            ("coding",   73),
            ("umbrella", 78)
        ]

        for s, expected in test_cases:
            result = hash_function(s)
            print(f"hash({s}) = {result}")
            self.assertEqual(result, expected)
            print(f"Test passed: hash('{s}') == {expected}\n")

        print("All hash_function(s) tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestHashFunction))

test_hash_strings (__main__.TestHashFunction.test_hash_strings) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation for hash_function(s)...

hash(hello) = 17
Test passed: hash('hello') == 17

hash(world) = 34
Test passed: hash('world') == 34

hash(python) = 91
Test passed: hash('python') == 91

hash(hash) = 15
Test passed: hash('hash') == 15

hash(coding) = 73
Test passed: hash('coding') == 73

hash(umbrella) = 78
Test passed: hash('umbrella') == 78

All hash_function(s) tests completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 2: Expanded Hash Function**

- **Definition**  

    - A verbose variant that prints each letter update step.

    - The formula remains identical, but each iteration reveals the current `hashval`, allowing observation of how ASCII codes and multiplication by 31 transform the sum.

- **Implementation Reason**  

    - The expanded output is used to show the break down each part of the process for clarity.


In [122]:
def hash_function_expanded(s):
    
    hashValue = 0
    print(f"\nHashing string '{s}':")
    print("-" * 77)

    for index, char in enumerate(s):
        ascii_value = ord(char)
        print(f"Step {index + 1}: char = '{char}' with an ASCII value of {ascii_value}, previous hash was {hashValue}")
        print(f"\thash = {ascii_value} + 31 * {hashValue}")
        hashValue = ascii_value + 31 * hashValue
        print(f"\tNew hash value after processing '{char}: {hashValue}")

    final_hash_value = hashValue % 101
    print(f"\nFinal hash value after modulo 101: {final_hash_value}")
    print("-" * 77)

    return final_hash_value

#### **Test Class**: `TestExpandedHashFunction`  

- **Method**
    - Runs the same strings tested in `TestHashFunction`.  

- **Verification**
    - Ensures the final hash is identical to that produced by `hash_function`, while intermediate printed values illustrate the accumulation process.  

- **Outcome**
    - Demonstrates each addition and multiplication step, clarifying how polynomial rolling accumulation evolves.

In [123]:
class TestExpandedHashFunction(unittest.TestCase):
    def test_hash_function_expanded(self):
        print("\nStarting validation for hash_function_expanded(s)...\n")

        test_cases = [
            ("hello",    17),
            ("world",    34),
            ("python",   91),
            ("hash",     15),
            ("coding",   73),
            ("umbrella", 78),
            ("rust",     17)
        ]

        for s, expected in test_cases:
            result = hash_function_expanded(s)
            self.assertEqual(result, expected)
            print(f"Test passed: hash('{s}') == {expected}\n")

        print("All hash_function_expanded(s) tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestExpandedHashFunction))


test_hash_function_expanded (__main__.TestExpandedHashFunction.test_hash_function_expanded) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation for hash_function_expanded(s)...


Hashing string 'hello':
-----------------------------------------------------------------------------
Step 1: char = 'h' with an ASCII value of 104, previous hash was 0
	hash = 104 + 31 * 0
	New hash value after processing 'h: 104
Step 2: char = 'e' with an ASCII value of 101, previous hash was 104
	hash = 101 + 31 * 104
	New hash value after processing 'e: 3325
Step 3: char = 'l' with an ASCII value of 108, previous hash was 3325
	hash = 108 + 31 * 3325
	New hash value after processing 'l: 103183
Step 4: char = 'l' with an ASCII value of 108, previous hash was 103183
	hash = 108 + 31 * 103183
	New hash value after processing 'l: 3198781
Step 5: char = 'o' with an ASCII value of 111, previous hash was 3198781
	hash = 111 + 31 * 3198781
	New hash value after processing 'o: 99162322

Final hash value after modulo 101: 17
-----------------------------------------------------------------------------
Test passed: hash('hello') == 17




<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 3: Reasons For Using 31 and 101**

- **Why 31?**

    - It is a **prime number**, this is important because it helps reduce the chance of **collisions** when hashing strings by distributing hash values more evenly accross the hash table. 

    - A **collision** occurs when two different inputs produce the same hash value. 

    - The multiplication using 31 makes the hash values less predictable and spread out over a wider range, minimising **clustering**.

    - 31 is also a small number, reducing the likelyhood of integer overflow in C due to it having fixed sized integers (such as 32 or 64-bit). 
    
    - However, since this has been converted to Python this is no longer an issue as it automatically swtiches from fixed-sized intgers to arbitrary-precision integers when needed. 
    
    - It is still used here though to maintain consistancy between languages and to prevent performance slowdown which can still happen with extremely large numbers

    - In C, multiplying by 31 can be optimised by compilers as a simple calculation it provides of **(hashval << 5) - hashval** enables the efficient use of bit-shifting and subtraction instead of pure multiplication. 
    
    - Once again, however, this isn't a significant issue in Python as it handles operations like multiplication and bit-shifting at a higher level so there is no significant performance difference between them.

- **Why 101?**

    - It is also a **prime number**, in this case used in the modulo operation to limit the hash values to be within a specific range, in this case 0 to 100.

    - Similar to 31, it being a prime number helps **distribute** the hash values more evenly across this range.

    - Using 101 also helps prevent the **clustering** of hash values, this can occur if the modulo base has **common factors** with the data. 
    
    - Examples of these factors are if the data contains patterns divisible by the modulo base. 101 **doesn't** share common factors with most data patterns, helping to greatly negate this risk.

## **Task 3 - SHA256**

**Purpose**  

- This task involves calculating the padding that would be added to a file’s contents before hashing it with SHA-256, in accordance with the specification defined in [FIPS PUB 180-4](https://doi.org/10.6028/NIST.FIPS.180-4). 

- Proper message padding is critical to the security and correct functioning of hash functions, as it ensures messages are interpreted consistently by all implementations.

**Cryptographic Context**  

- In the SHA-2 family of cryptographic hash functions, padding ensures that the total message length (in bits) is a multiple of 512. 

- The padding also embeds the original message length, which prevents length extension attacks and maintains integrity.

**Overview**  

- The implemented function performs the following:

    - Reads the input file’s raw bytes.

    - Appends a `1` bit (`0x80`) followed by enough `0` bits to make the total length 64 bits short of a multiple of 512.

    - Appends the original message length as a 64-bit big-endian integer.

    - Outputs the final padding in hexadecimal format for inspection or testing.

    - The test verifies this process using the `"abc"` string, which is the canonical example from FIPS 180-4 Appendix B.1.


### **Step 1: Create Temporary File**

- **Definition**  

    - Creates a temporary binary file containing the ASCII string `"abc"` using the `tempfile` module. 

    - The file simulates a raw input source used in cryptographic operations.


In [179]:
with tempfile.NamedTemporaryFile(delete=False, mode="wb") as temp_file:
    temp_file.write(b"abc")
    temp_file_path = temp_file.name

#### **Test**

- Confirms that the file exists, and states where it is stored.

In [180]:
print(f"Temporary file created at: {temp_file_path}")

Temporary file created at: C:\Users\melgo\AppData\Local\Temp\tmpboieivd9


### **Step  2: Read Temporary File**

- **Definition**  

    - Reads the contents of the temporary file in binary mode and stores it as a byte array for further processing.

In [181]:
with open(temp_file_path, "rb") as file:
    data = file.read()

#### **Test Class: `TestBinaryFileContents`**

- **Test Approach**  

    - The file is opened using `open(path, "rb")` and read into a variable using `.read()`. 

    - Each byte is then printed in binary using an 8-bit format.

- **Verification** 

    - The test confirms that the binary output is `01100001 01100010 01100011`, which corresponds to the ASCII values of `"abc"`.

- **Outcome**

    - The file is successfully read and the binary content matches the expected input.

In [183]:
class TestBinaryFileContents(unittest.TestCase):
    def test_binary_representation(self):
        print("\nStarting validation of binary contents from file...\n")

        bit_string = " ".join(f"{byte:08b}" for byte in data)
        joined_bits = "".join(bit_string)
        print("Binary contents of the file:", joined_bits)

        expected = "01100001 01100010 01100011"
        self.assertEqual(bit_string, expected)
        print("Test passed: binary content matches expected bit pattern\n")

        print("Binary file content test completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestBinaryFileContents))


test_binary_representation (__main__.TestBinaryFileContents.test_binary_representation) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation of binary contents from file...

Binary contents of the file: 01100001 01100010 01100011
Test passed: binary content matches expected bit pattern

Binary file content test completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 3: Calculate the Original Length of File Data in Bits**

- **Definition**  

    - Multiplies the byte length of the file data by 8 to calculate the original message length in bits. 
    
    - This is needed because SHA-256 requires the original bit length to be appended to the padded message, and the length must be expressed in bits, not bytes.


In [184]:
num_bytes = len(data) 
original_length_bits = num_bytes * 8

#### **Test Class: `TestOriginalLengthInBits`**

- **Test Approach**  

    - The length of the byte array is calculated using `len(data)`, and then multiplied by 8 to yield the total number of bits.

- **Verification**  

    - The test confirms that the result is 24 bits for the 3-byte message (`3 × 8 = 24`).

- **Outcome**  

    - The original length is correctly calculated as 24 bits.

In [185]:
class TestOriginalLengthInBits(unittest.TestCase):
    def test_original_length_bits(self):
        print("\nStarting validation of original bit length calculation...\n")

        num_bytes = len(data)
        original_length_bits = num_bytes * 8

        bit_string = " ".join(f"{byte:08b}" for byte in data)

        print(f"Original length in bits: {num_bytes} × 8 = {original_length_bits} (Sum of: {bit_string})")

        self.assertEqual(original_length_bits, 24)
        print("Test passed: calculated bit length is correct\n")

        print("Bit length test completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestOriginalLengthInBits))


test_original_length_bits (__main__.TestOriginalLengthInBits.test_original_length_bits) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation of original bit length calculation...

Original length in bits: 3 × 8 = 24 (Sum of: 01100001 01100010 01100011)
Test passed: calculated bit length is correct

Bit length test completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 4: Append 1 Bit (0x80) Onto End of Data**

- **Definition**  

    - Appends a 1-bit suffix (represented by the byte `0x80`) to the end of the message. 
    
    - This padding is required by SHA-256 to signal the end of the original message and the start of the padding process.

In [186]:
# Marks the end of the file data
padded_message = data + b'\x80'

#### **Test Class: `TestAppendOneBit`**

- **Test Approach**  

    - The byte `0x80` is appended to the original message using `data + b'\x80'`. 

    - The result is converted to binary and compared to the expected pattern.

- **Verification**  

    - The binary sequence is verified to be `01100001 01100010 01100011 10000000`.

- **Outcome**  

    - The 1-bit padding is correctly applied and the message length increases from 3 to 4 bytes.


In [132]:
class TestAppendOneBit(unittest.TestCase):
    def test_append_one_bit(self):
        print("\nStarting validation of appending 1-bit (0x80)...\n")

        padded_message = data + b'\x80'
        bit_string = " ".join(f"{b:08b}" for b in padded_message)

        print(f"After adding 1-bit: {bit_string}")


        expected = "01100001 01100010 01100011 10000000"
        self.assertEqual(bit_string, expected)
        print("Test passed: 1-bit (0x80) correctly appended\n")

        print("Append 1-bit test completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestAppendOneBit))

test_append_one_bit (__main__.TestAppendOneBit.test_append_one_bit) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation of appending 1-bit (0x80)...

After adding 1-bit: 01100001 01100010 01100011 10000000
Test passed: 1-bit (0x80) correctly appended

Append 1-bit test completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **5: Calculate Zero Padding**

- **Definition**  

    - Calculates and appends the necessary number of `0x00` bytes to bring the message length to 56 bytes. 
    
    - This ensures that there is enough space for the 64-bit message length at the end of the padded message, allowing the message to conform to the 512-bit block size required by SHA-256.


In [133]:
zero_padding_length = (56 - (len(padded_message) % 64)) % 64

padded_message += b'\x00' * zero_padding_length

#### **Test Class: `TestZeroPadding`**

- **Test Approach**  
    
    - The required padding length is calculated.
    
    - Zero bytes are added.

- **Verification**  

    - The test confirms that exactly 52 bytes of `0x00` are appended, resulting in a message of 56 bytes.

- **Outcome**  

    - Zero padding is correctly calculated and applied.


In [134]:
class TestZeroPadding(unittest.TestCase):
    def test_zero_padding(self):
        print("\nStarting validation of zero padding to 56-byte boundary...\n")

        # Rebuild padded_message from original data each time
        padded_message = data + b'\x80'
        zero_padding_length = (56 - (len(padded_message) % 64)) % 64
        padded_message += b'\x00' * zero_padding_length
        bit_string = " ".join(f"{b:08b}" for b in padded_message)

        print(f"After zero padding: {bit_string}")
        print(f"Padding length: {zero_padding_length} bytes")

        expected_padding_length = 52
        self.assertEqual(zero_padding_length, expected_padding_length)
        self.assertEqual(len(padded_message), 56)
        print("Test passed: zero padding applied correctly\n")

        print("Zero padding test completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestZeroPadding))

test_zero_padding (__main__.TestZeroPadding.test_zero_padding) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation of zero padding to 56-byte boundary...

After zero padding: 01100001 01100010 01100011 10000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
Padding length: 52 bytes
Test passed: zero padding applied correctly

Zero padding test completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 6: Append the Original Message Length to End of Padded Message**

- **Definition**  

    - Appends the original message length (in bits) as a 64-bit big-endian integer to the end of the padded message. 
    
    - This is necessary because the SHA-256 algorithm processes messages in 512-bit blocks, and the final 64 bits are reserved for the original message length. 
    
    - This ensures that the total length of the padded message is a multiple of 512 bits, which is crucial for the hashing process and helps maintain the message’s integrity throughout the compression function.

In [135]:
padded_message += original_length_bits.to_bytes(8, 'big')

#### **Test Class: TestFinalPaddedMessage**

- **Test Approach**  

    - The value `24` (original bit length) is converted and appended to the padded message.

- **Verification**  

    - The final message length is verified to be 64 bytes, and the last 8 bytes match 0000000000000018.

- **Outcome**  

    - The final message is correctly padded and formatted as a 512-bit block.


In [136]:
class TestFinalPaddedMessage(unittest.TestCase):
    def test_append_length_block(self):
        print("\nStarting validation of final padded message with 64-bit length...\n")

        # Reconstruct everything from scratch for a clean test
        padded_message = data + b'\x80'
        zero_padding_length = (56 - (len(padded_message) % 64)) % 64
        padded_message += b'\x00' * zero_padding_length

        original_length_bits = len(data) * 8
        padded_message += original_length_bits.to_bytes(8, 'big')

        bit_string = " ".join(f"{b:08b}" for b in padded_message)
        print(f"Final padded message: {bit_string}")

        self.assertEqual(len(padded_message), 64)
        self.assertEqual(padded_message[-8:], original_length_bits.to_bytes(8, 'big'))
        print("Test passed: final message is correctly padded and length block is correct\n")

        print("Final padding test completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestFinalPaddedMessage))

test_append_length_block (__main__.TestFinalPaddedMessage.test_append_length_block) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation of final padded message with 64-bit length...

Final padded message: 01100001 01100010 01100011 10000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00011000
Test passed: final message is correctly padded and length block is correct

Final padding test completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 7: Extract and Display Only the Padding**

- **Definition**

    - Extracts and displays only the padding portion of the message (excluding the original data), formatted as hexadecimal for inspection. 
    
    - This allows verification that the padding was applied correctly and matches the expected format.


In [137]:
padding_hex = padded_message[len(data):]

#### **Test Class: `TestSha256PaddingHex`**

- **Test Approach**  

    - The original message bytes are sliced out and the remaining padding is converted to a hex string for inspection.

- **Verification**  

    - The hex output is confirmed to be 80, followed by 52 00s, and ending with 00 00 00 00 00 00 00 18.

- **Outcome**  

    - Padding structure matches the expected SHA-256 format exactly.

In [138]:
class TestSha256PaddingHex(unittest.TestCase):
    def test_padding_hex_output(self):
        print("\nStarting validation of SHA-256 padding in hex format...\n")

        # Rebuild full padded message
        padded_message = data + b'\x80'
        zero_padding_length = (56 - (len(padded_message) % 64)) % 64
        padded_message += b'\x00' * zero_padding_length

        original_length_bits = len(data) * 8
        padded_message += original_length_bits.to_bytes(8, 'big')

        # Extract only the padding part (everything after the original data)
        padding_hex = padded_message[len(data):]

        hex_output = " ".join(f"{b:02X}" for b in padding_hex)
        print(f"SHA-256 Padding (Hex): {hex_output}")

        expected_hex = (
            "80 " + "00 " * 52 + "00 00 00 00 00 00 00 18"
        ).strip()

        self.assertEqual(hex_output, expected_hex)
        print("Test passed: SHA-256 padding hex output is correct\n")

        print("SHA-256 padding hex test completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestSha256PaddingHex))


test_padding_hex_output (__main__.TestSha256PaddingHex.test_padding_hex_output) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation of SHA-256 padding in hex format...

SHA-256 Padding (Hex): 80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18
Test passed: SHA-256 padding hex output is correct

SHA-256 padding hex test completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 8. Delete the Temporary File**

- **Definition**  

    - Deletes the temporary file from disk after all padding operations are completed. 

    - This step ensures that there are no leftover resources, adhering to best practices for resource management.

    - The file is deleted using `os.remove(path)`. 
    
    - File existence is confirmed before and after the call.

In [139]:
print(f"File exists before deletion: {os.path.exists(temp_file_path)}")

try:
    os.remove(temp_file_path)

    # Check if the file still exists
    if os.path.exists(temp_file_path):
        print("\nError: File was NOT deleted!")
    else:
        print("\nTemporary file deleted successfully.")

except FileNotFoundError:
    print("\nWarning: File already deleted or does not exist.")
except Exception as e:
    print(f"\nUnexpected error: {e}")

print(f"\nFile exists after deletion: {os.path.exists(temp_file_path)}")

File exists before deletion: True

Temporary file deleted successfully.

File exists after deletion: False


## **Task 4 - Prime Numbers**

**Purpose**

- This task implements two prime number algorithms: Trial Division and the Sieve of Atkin. 

- Prime Numbers are numbers that cannot be exactly divided (i.e. without a remainder) by any whole number other than itself and 1.

- The following two approaches are implemented and tested:

    - **Trial Division** - Confirms whether a single integer is prime by checking divisibility against a reduced range of candidate factors, skipping obvious non-primes.

    - **Sieve of Atkin** - Generates a full list of prime numbers up to a given limit by marking candidates with quadratic rules and eliminating square multiples.

- Each algorithm is tested for correctness using unit tests that validate individual steps and expected outputs.

- These techniques enable deterministic identification and collection of prime numbers, which are critical in mathematical computation and cryptographic protocols.

**Cryptographic Context**

- Prime numbers are a fundamental building block in cryptographic systems. 

- Public-key cryptography relies on the generation of large prime numbers to create secure and unpredictable key pairs. 

- The difficulty of factoring large composites directly depends on the ability to efficiently generate and verify prime numbers.

- Primality testing and prime enumeration ensure:

    - Secure key generation through reliable identification of large prime values.

    - Cryptographic strength by avoiding weak or repeated factors.

    - Improved performance of cryptosystems relying on modular arithmetic with primes.

- This task validates the correctness and performance of both trial-based and sieve-based prime number techniques for cryptographic use.



### **Algorithm 1: Trial Division Algorithm**

#### **Step 1: `def is_prime(n)`**

- Definition

    - Defines a function that determines whether a given integer is prime using trial division. 
    
    - Numbers less than 2 are excluded, as neither 0 nor 1 are prime. 
    
    - The function first eliminates even numbers greater than 2 and multiples of 3, which are not prime by definition. 

    - It then checks for divisibility using a loop that tests numbers of the form 6k ± 1, based on the fact that all primes greater than 3 fall into that pattern. 
    
    - This reduces the number of checks required while still correctly identifying all primes. 
    
    - The loop runs only up to the square root of the input, since any larger factor would already have a corresponding smaller one.

    - This method works reliably for small to medium input sizes and ensures that composite numbers are accurately filtered out without redundant checks.




In [140]:
def is_prime(n):
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

#### **Step 2: `def first_n_primes_trial(n)`**

- Definition

    - Defines a function that generates the first N prime numbers using the `is_prime(n)` function. 
    
    - Starting from 2, it tests each number in order and appends confirmed primes to a list until it contains N elements.

    - This method guarantees correctness by individually verifying each candidate using a proven primality test. 
    
    - The output list grows sequentially, so the result is always in increasing order.
    
    - The approach is simple and reliable for generating relatively small sets of prime numbers and is useful for verifying the correctness of more advanced methods like sieves later in the task.

In [141]:
def first_n_primes_trial(n):
    primes = []
    num = 2
    while len(primes) < n:
        if is_prime(num):
            primes.append(num)
        num += 1
    return primes

#### **Test Class: `TestPrimeFunctions`**

- Test Approach

    - Two functions are tested. `is_prime(n)` is tested using known prime and non-prime values to confirm it correctly identifies valid and invalid inputs. 
    
    - `first_n_primes_trial(n)` is tested with small input values to verify that the returned lists contain the expected prime numbers with the correct length and ordering.

- Verification

    - `is_prime(n)` correctly returns True for primes and False for non-primes  

    - `first_n_primes_trial(n)` returns the correct prime sequences for small `n`

- Outcome

    - The test confirms that both `is_prime(n)` and `first_n_primes_trial(n)` behave as expected. 


In [187]:
class TestPrimeFunctions(unittest.TestCase):
    def test_is_prime(self):
        print("\nTesting is_prime(n)...\n")

        self.assertTrue(is_prime(2))
        print("Test passed: 2 is prime")
        self.assertTrue(is_prime(17))
        print("Test passed: 17 is prime")

        self.assertFalse(is_prime(1))
        print("Test passed: 1 is not prime")
        self.assertFalse(is_prime(100))
        print("Test passed: 100 is not prime")

        print("\nis_prime(n) tests completed successfully.\n")

    def test_first_n_primes_trial(self):
        print("\nTesting first_n_primes_trial(n)...\n")

        self.assertEqual(first_n_primes_trial(1), [2])
        print("Test passed: first_n_primes_trial(1) == [2]")

        self.assertEqual(first_n_primes_trial(3), [2, 3, 5])
        print("Test passed: first_n_primes_trial(3) == [2, 3, 5]")

        print("\nfirst_n_primes_trial(n) tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestPrimeFunctions))


test_first_n_primes_trial (__main__.TestPrimeFunctions.test_first_n_primes_trial) ... ok
test_is_prime (__main__.TestPrimeFunctions.test_is_prime) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK



Testing first_n_primes_trial(n)...

Test passed: first_n_primes_trial(1) == [2]
Test passed: first_n_primes_trial(3) == [2, 3, 5]

first_n_primes_trial(n) tests completed successfully.


Testing is_prime(n)...

Test passed: 2 is prime
Test passed: 17 is prime
Test passed: 1 is not prime
Test passed: 100 is not prime

is_prime(n) tests completed successfully.



<unittest.runner.TextTestResult run=2 errors=0 failures=0>

#### **Step 3: Execute Prime Sequence Generator for 1000 Primes**

- Definition

    - Executes the `first_n_primes_trial(1000)` function to generate the first 1000 prime numbers. 
    
    - The result is printed to verify that the function works as expected for a larger value of N.

    - This step validates the correctness and performance of the prime generator.
    
    - The resulting list contains all prime numbers in sequence from 2 up to the 1000th prime.

In [143]:
primes_output= first_n_primes_trial(1000)
print(primes_output)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 12

### **Sieve of Atkin Algorithm**

#### **Step 1: Implement Sieve of Atkin Algorithm**

- Definition

    - It creates an optimized algorithm for finding all prime numbers up to a specified limit.
    
    - It returns an empty list if the limit is less than 2, since there are no prime numbers below 2.
    
    - It initializes a sieve as a boolean array where all values are False (not prime) by default.
    
    - It explicitly marks 2 and 3 as prime (True) since they don't follow the patterns used by the main algorithm.
    
    - It instead of checking each number individually like Trial Division, it uses mathematical quadratic equations to mark numbers that could potentially be prime.
    
    - It only considers numbers as possible primes if they fit specific repeating patterns in modulo 12 arithmetic, which helps to quickly skip unnecessary checks.
    
    - **Formula 1**: `4x² + y² ≡ 1 or 5 mod 12` identifies numbers that have remainder 1 or 5 when divided by 12.
    
    - **Formula 2**: `3x² + y² ≡ 7 mod 12` identifies numbers that have remainder 7 when divided by 12.
    
    - **Formula 3**: `3x² − y² ≡ 11 mod 12` (only if `x > y`) identifies numbers that have remainder 11 when divided by 12, only when x > y.
    
    - It toggles the primality status of each number that fits these patterns (from False to True or True to False).
    
    - It eliminates false positives by marking all multiples of squares of identified primes as non-prime.
    
    - When this filtering is completed, the remaining marked numbers are guaranteed to be prime.
    
    - It returns a list containing all numbers that remain marked as prime after the sieving process.

In [144]:
def sieve_of_atkin(limit):
    if limit < 2:
        return []

    ## Initialise the sieve and create list of False values that assumes all numbers are not prime
    sieve = [False] * (limit + 1)
    sieve[2] = sieve[3] = True

    ## Use Quadratic Equations to find potential prime numbers
    for x in range(1, int(limit**0.5) + 1):
        for y in range(1, int(limit**0.5) + 1):
            
            ## Formula 1: Checks if number is modulo 12 = 1 or 5
            n = (4 * x * x) + (y * y)
            if n <= limit and (n % 12 == 1 or n % 12 == 5):
                sieve[n] = not sieve[n]

            ## Formula 2: Checks if number is modulo 12 ≡ 7
            n = (3 * x * x) + (y * y)
            if n <= limit and n % 12 == 7:
                sieve[n] = not sieve[n]

            ## Formula 3: Checks if number is modulo 12 ≡ 11
            n = (3 * x * x) - (y * y)
            if x > y and n <= limit and n % 12 == 11:
                sieve[n] = not sieve[n]

    ## Mark all multiples of known primes as non-prime to eliminate false positives
    for num in range(5, int(limit**0.5) + 1):
        if sieve[num]:
            for multiple in range(num * num, limit + 1, num * num):
                sieve[multiple] = False

    return [num for num in range(limit + 1) if sieve[num]]


#### **Test Class `TestSieveOfAtkinFormulas`**

- Test Approach

    - This test class verifies the functionality of the Sieve of Atkin algorithm. 

    - It confirms that the sieve correctly identifies primes for small values of `n` (e.g., 5, 7, 11).

    - It verifies that the sieve eliminates non-prime numbers, especially composite numbers like squares of primes, which is critical for ensuring the algorithm’s accuracy and efficiency.  

- Verification

    - The sieve correctly identifies primes up to a given limit.

    - The sieve accurately eliminates non-prime numbers, such as squares of primes.

- Outcome

    - All tests pass, confirming that the Sieve of Atkin algorithm works as expected and correctly identifies primes.


In [ ]:
class TestSieveOfAtkinFormulas(unittest.TestCase):
    def test_formula_1_mod_12_1_or_5(self):
        print("\nTesting Formula 1: 4x² + y² ≡ 1 or 5 (mod 12)...\n")
        primes = sieve_of_atkin(5)
        self.assertIn(5, primes)
        print("Test passed: 5 correctly included by Formula 1")

    def test_formula_2_mod_12_7(self):
        print("\nTesting Formula 2: 3x² + y² ≡ 7 (mod 12)...\n")
        primes = sieve_of_atkin(7)
        self.assertIn(7, primes)
        print("Test passed: 7 correctly included by Formula 2")

    def test_formula_3_mod_12_11(self):
        print("\nTesting Formula 3: 3x² − y² ≡ 11 (mod 12)...\n")
        primes = sieve_of_atkin(11)
        self.assertIn(11, primes)
        print("Test passed: 11 correctly included by Formula 3")

    def test_false_positives_eliminated(self):
        print("\nTesting elimination of false positives (non-primes marked off)...\n")
        primes = sieve_of_atkin(50)
        self.assertNotIn(49, primes)  # 49 = 7² should be filtered
        self.assertNotIn(9, primes)   # 9 = 3²
        print("Test passed: composite squares eliminated as expected")

        print("\nsieve_of_atkin formula-specific tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestSieveOfAtkinFormulas))


test_false_positives_eliminated (__main__.TestSieveOfAtkinFormulas.test_false_positives_eliminated) ... ok
test_formula_1_mod_12_1_or_5 (__main__.TestSieveOfAtkinFormulas.test_formula_1_mod_12_1_or_5) ... ok
test_formula_2_mod_12_7 (__main__.TestSieveOfAtkinFormulas.test_formula_2_mod_12_7) ... ok
test_formula_3_mod_12_11 (__main__.TestSieveOfAtkinFormulas.test_formula_3_mod_12_11) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.005s

OK



Testing elimination of false positives (non-primes marked off)...

Test passed: composite squares eliminated as expected

sieve_of_atkin formula-specific tests completed successfully.


Testing Formula 1: 4x² + y² ≡ 1 or 5 (mod 12)...

Test passed: 5 correctly included by Formula 1

Testing Formula 2: 3x² + y² ≡ 7 (mod 12)...

Test passed: 7 correctly included by Formula 2

Testing Formula 3: 3x² − y² ≡ 11 (mod 12)...

Test passed: 11 correctly included by Formula 3


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

#### **Step 2: Execute Sieve of Atkin for 1000 Primes**

- Definition

   - Executes the Sieve of Atkin algorithm to generate prime numbers, targeting the first 1000 primes.
   
   - Uses 8000 as the upper limit to ensure capturing at least 1000 prime numbers, since the 1000th prime is approximately 7919.
   
   - Applies list slicing with [:1000] to extract exactly the first 1000 primes from the generated list.
   
   - Prints the resulting list to display all 1000 prime numbers generated by the Sieve of Atkin algorithm.
   
   - This approach allows comparison with the Trial Division method to verify that both algorithms produce identical results for the first 1000 primes.
   
   - The Sieve of Atkin typically produces the result much faster than Trial Division, especially for larger ranges of prime numbers.

In [146]:
primes_atkin = sieve_of_atkin(8000)[:1000]
print(primes_atkin)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 12

### **Findings: Prime Number Algorithms**

- Both the Trial Division and Sieve of Atkin methods were validated to produce identical prime sequences up to the 1000th prime, confirming consistency and correctness.

- **Comparison**:

  | Aspect               | Trial Division              | Sieve of Atkin                   |
  |----------------------|-----------------------------|----------------------------------|
  | Method Type          | Incremental check per value | Bulk sieve using math patterns   |
  | Performance          | Slower for large ranges     | Optimized for large ranges       |
  | Complexity           | Simple to understand        | Mathematically more complex      |
  | Use Case             | Small sets or validation    | High-performance prime generation |


### **Conclusion**:
  - For small inputs or educational purposes, **Trial Division** offers clarity and simplicity.  
  - For generating large numbers of primes efficiently, the **Sieve of Atkin** is the superior choice due to its significantly better runtime performance.

## **Task 5 - Roots**

**Purpose**

- This task extracts the 32-bit binary representation of the fractional part of square roots of prime numbers.
- It defines a function to isolate and convert the fractional part of any floating-point value into a fixed-size binary integer.
- It uses the Sieve of Atkin from Task 4 to generate the first 100 prime numbers.
- For each prime, the square root is calculated and its fractional part is encoded into a 32-bit binary integer using the defined function.
- This allows binary data to be derived from mathematical constants, which is useful in deterministic random generation and cryptographic precomputation.

**Cryptographic Context**

- Fractional binary representations of irrational numbers (like square roots of primes) are used in cryptography to derive constants that are hard to reverse-engineer.
- SHA-256, for example, uses the fractional parts of square roots of primes to define initial hash values.
- Extracting these representations helps demonstrate how deterministic constants can be produced from well-known but non-repeating values.
- This supports the design of cryptographic systems where reproducible, high-entropy values are required.

### **Step 1: Extract Fractional Bits from Real Numbers**

- **Definition**

    - Defines a function that isolates the fractional part of a float and extracts a specified number of bits from it.

    - Uses iterative doubling to shift fractional digits into binary form.

    - Accumulates the result as a binary integer by checking whether each successive bit is 1 or 0.

    - Works for any real number and customizable bit lengths.

    - Useful for encoding non-repeating binary patterns from mathematical constants.

In [148]:
def get_fractional_bits(value, bits=32):
    """Extracts the first `bits` binary digits of the fractional part of a number."""
    fractional_part = value - math.floor(value) 
    result = 0
    for i in range(bits):
        fractional_part *= 2
        bit = int(fractional_part)
        result = (result << 1) | bit
        fractional_part -= bit
    return result

#### **Test Class: `TestGetFractionalBits`**

- **Test Approach**
    - Evaluates the function against different categories of inputs: 0.0, whole numbers, fractions, irrational numbers, and limited-bit precision cases.

    - Each case includes an expected output or property check.

- **Verification**
    - 0.0 and integers have no fractional component and should return 0.

    - 0.5 returns a result with only the highest bit set.

    - Square roots and irrational constants return valid 32-bit binary representations.

    - Outputs are checked for bit range and type.

- **Outcome**
    - Function correctly handles all input types and bit sizes.
    
    - Demonstrates the function’s ability to reliably extract deterministic bit patterns.

In [149]:
class TestGetFractionalBits(unittest.TestCase):
    def test_zero(self):
        print("\nTesting get_fractional_bits(0.0)...")
        result = get_fractional_bits(0.0)
        print(f"Output: {result:032b}")
        self.assertEqual(result, 0)
        print("Test passed: fractional bits of 0.0 == 0")

    def test_integer_input(self):
        print("\nTesting get_fractional_bits(7)...")
        result = get_fractional_bits(7)
        print(f"Output: {result:032b}")
        self.assertEqual(result, 0)
        print("Test passed: fractional bits of 7 == 0")

    def test_half(self):
        print("\nTesting get_fractional_bits(0.5)...")
        result = get_fractional_bits(0.5)
        expected = int('1' + '0'*31, 2)
        print(f"Output: {result:032b}")
        self.assertEqual(result, expected)
        print("Test passed: fractional bits of 0.5 == 1000...0")

    def test_sqrt_2(self):
        print("\nTesting get_fractional_bits(math.sqrt(2))...")
        value = math.sqrt(2)
        result = get_fractional_bits(value)
        print(f"Output: {result:032b}")
        self.assertIsInstance(result, int)
        self.assertLess(result, 2**32)
        print("Test passed: fractional bits of √2 is a valid 32-bit integer")

    def test_custom_bits(self):
        print("\nTesting get_fractional_bits(math.pi, bits=8)...")
        value = math.pi
        result = get_fractional_bits(value, bits=8)
        print(f"Output: {result:08b}")
        self.assertGreaterEqual(result, 0)
        self.assertLess(result, 256)
        print("Test passed: fractional bits of π (8 bits) is within range 0-255")

        print("\nget_fractional_bits tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestGetFractionalBits))

test_custom_bits (__main__.TestGetFractionalBits.test_custom_bits) ... ok
test_half (__main__.TestGetFractionalBits.test_half) ... ok
test_integer_input (__main__.TestGetFractionalBits.test_integer_input) ... ok
test_sqrt_2 (__main__.TestGetFractionalBits.test_sqrt_2) ... ok
test_zero (__main__.TestGetFractionalBits.test_zero) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.004s

OK



Testing get_fractional_bits(math.pi, bits=8)...
Output: 00100100
Test passed: fractional bits of π (8 bits) is within range 0-255

get_fractional_bits tests completed successfully.


Testing get_fractional_bits(0.5)...
Output: 10000000000000000000000000000000
Test passed: fractional bits of 0.5 == 1000...0

Testing get_fractional_bits(7)...
Output: 00000000000000000000000000000000
Test passed: fractional bits of 7 == 0

Testing get_fractional_bits(math.sqrt(2))...
Output: 01101010000010011110011001100111
Test passed: fractional bits of √2 is a valid 32-bit integer

Testing get_fractional_bits(0.0)...
Output: 00000000000000000000000000000000
Test passed: fractional bits of 0.0 == 0


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

### **Step 2: Generate First 100 Prime Numbers**

- **Definition**

    - Reuses the `primes_atkin` list from Task 4, no recomputation is needed as primes were already generated.

    - Uses slicing to extract the first 100 primes from the precomputed sequence.

In [150]:
# Using the Atkin Sieve from task 4 to generate the first 100 prime numbers
first_100_primes = primes_atkin[:100] 

### **Step 3: Compute Fractional Bits of √p for First 100 Primes**

- **Definition**

    - It iterates over the first 100 prime numbers.

    - For each prime, it calculates its square root.

    - It extracts the first 32 bits of the fractional part of the square root.

    - It stores each 32-bit result in a list.

In [151]:
# Compute and store the 32-bit fractional binary representation of the square roots
sqrt_fractional_bits = []
for prime in first_100_primes:
    sqrt_val = math.sqrt(prime)
    bits = get_fractional_bits(sqrt_val, bits=32)
    sqrt_fractional_bits.append(bits)

#### **Test Class: `TestSqrtFractionalBitsExtraction`**

- **Test Approach**
    - Validates the 32-bit fractional bit extraction of prime square root for the first 100 prime numbers.

    - Ensures length, type, and correctness of output values.

    - Verifies deterministic behavior across repeated runs.

- **Verification**
    - Confirms output list has exactly 100 results.

    - Checks that all results are 32-bit integers within valid bounds.

    - Validates that no entries are `None` or `NaN`.

    - Confirms output is deterministic when the same input is used.

- **Outcome**
    - All tests pass.
    
    - Function is confirmed to produce valid, reproducible results for the square root bit extraction.

In [192]:
class TestSqrtFractionalBitsExtraction(unittest.TestCase):

    def test_first_100_primes_sqrt_bits(self):
        first_100_primes = primes_output[:100]

        print("\nStarting validation for 32-bit fractional extraction from square roots of the first 100 prime numbers...\n")

        # Compute the fractional bits
        sqrt_fractional_bits = []
        for prime in first_100_primes:
            sqrt_val = math.sqrt(prime)
            bits = get_fractional_bits(sqrt_val, bits=32)
            sqrt_fractional_bits.append(bits)

        # Test 1: Correct number of results
        print("Running Test 1: Check result length == 100...")
        self.assertEqual(len(sqrt_fractional_bits), 100)
        print("Test passed: Output list contains 100 entries.\n")

        # Test 2: All values are valid 32-bit integers
        print("Running Test 2: Validate each output is a valid 32-bit integer...")
        for bits in sqrt_fractional_bits:
            self.assertIsInstance(bits, int)
            self.assertGreaterEqual(bits, 0)
            self.assertLess(bits, 2**32)
        print("Test passed: All entries are valid 32-bit integers.\n")

        # Test 3: No None or NaN values
        print("Running Test 3: Ensure no None or NaN values...")
        for bits in sqrt_fractional_bits:
            self.assertIsNotNone(bits)
            self.assertFalse(math.isnan(bits))
        print("Test passed: No None or NaN values in the output.\n")

        # Test 4: Deterministic output
        print("Running Test 4: Confirm deterministic output...")
        for prime in first_100_primes:
            sqrt_val = math.sqrt(prime)
            bits1 = get_fractional_bits(sqrt_val, bits=32)
            bits2 = get_fractional_bits(sqrt_val, bits=32)
            self.assertEqual(bits1, bits2)
        print("Test passed: Results are reproducible (deterministic).\n")

        # Test 5: Validate bit correctness for known input
        print("Running Test 5: Validate bit correctness for known input (√2)...")
        sqrt_2 = math.sqrt(2)
        result = get_fractional_bits(sqrt_2, bits=32)
        expected = int('01101010000010011110011001100111', 2)
        self.assertEqual(result, expected)
        print("Test passed: Fractional bits of √2 match expected binary value.\n")

        print("\nAll sqrt(prime) fractional bit extraction tests completed successfully.\n")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestSqrtFractionalBitsExtraction))


test_first_100_primes_sqrt_bits (__main__.TestSqrtFractionalBitsExtraction.test_first_100_primes_sqrt_bits) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.010s

OK



Starting validation for 32-bit fractional extraction from square roots of the first 100 prime numbers...

Running Test 1: Check result length == 100...
Test passed: Output list contains 100 entries.

Running Test 2: Validate each output is a valid 32-bit integer...
Test passed: All entries are valid 32-bit integers.

Running Test 3: Ensure no None or NaN values...
Test passed: No None or NaN values in the output.

Running Test 4: Confirm deterministic output...
Test passed: Results are reproducible (deterministic).

Running Test 5: Validate bit correctness for known input (√2)...
Test passed: Fractional bits of √2 match expected binary value.


All sqrt(prime) fractional bit extraction tests completed successfully.



<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 4: Display Fractional Bit Output for Selected Primes**

- **Definition**

    - It pairs each prime number with the 32-bit binary representation of the fractional part of its square root.

    - It displays only the first 10 and last 10 (prime, bits) pairs for clarity.

In [191]:
# Create (prime, bits) string pairs
pairs = [(p, f"{get_fractional_bits(math.sqrt(p), bits=32):032b}") for p in first_100_primes]

# Print first 10
print("First 10 primes with fractional bits of their square roots:\n")
for p, b in pairs[:10]:
    print(f"Prime: {p} -> Bits: {b}")

# Ellipsis separator
print("\n...\n")

# Print last 10
print("Last 10 primes with fractional bits of their square roots:\n")
for p, b in pairs[-10:]:
    print(f"Prime: {p} -> Bits: {b}")


First 10 primes with fractional bits of their square roots:

Prime: 2 -> Bits: 01101010000010011110011001100111
Prime: 3 -> Bits: 10111011011001111010111010000101
Prime: 5 -> Bits: 00111100011011101111001101110010
Prime: 7 -> Bits: 10100101010011111111010100111010
Prime: 11 -> Bits: 01010001000011100101001001111111
Prime: 13 -> Bits: 10011011000001010110100010001100
Prime: 17 -> Bits: 00011111100000111101100110101011
Prime: 19 -> Bits: 01011011111000001100110100011001
Prime: 23 -> Bits: 11001011101110111001110101011101
Prime: 29 -> Bits: 01100010100110100010100100101010

...

Last 10 primes with fractional bits of their square roots:

Prime: 467 -> Bits: 10011100001101001111000001100010
Prime: 479 -> Bits: 11100010110101010110010011000100
Prime: 487 -> Bits: 00010001011011010111010111111101
Prime: 491 -> Bits: 00101000100101001100000100000111
Prime: 499 -> Bits: 01010110100110110101100011000110
Prime: 503 -> Bits: 01101101011110110011100100111001
Prime: 509 -> Bits: 1000111110011111100

### **Conclusion: Square Root Fractional Bit Extraction**

- This task successfully demonstrated how to extract deterministic 32-bit binary patterns from the fractional parts of square roots of prime numbers.

- The `get_fractional_bits` function was tested and verified for correctness, edge cases, and reproducibility.

- The first 100 prime numbers were reused from the Sieve of Atkin implementation in Task 4.

- The extracted bit patterns were validated through automated tests and visually confirmed using printed output.

- This method parallels how constants are generated in cryptographic systems like SHA-256, where square root and cube root fractional parts of primes are used to initialize hash functions.

- The process is mathematically sound, reproducible, and provides entropy suitable for cryptographic applications.

- Extracting fractional binary patterns from irrational roots is a reliable technique for deterministic constant generation in security systems.

## **Task 6 - Proof of Work**

**Purpose**

- This task simulates a basic proof-of-work mechanism by analyzing the SHA-256 hashes of English dictionary words.

- The goal is to find which word(s) produce the most leading zero bits in their 256-bit SHA-256 hash.

- A dictionary file is loaded and processed word by word to evaluate hash properties.

- The task mirrors the concept of mining in cryptocurrencies, where higher leading zero counts indicate greater computational rarity.

**Cryptographic Context**

- Proof-of-work systems rely on cryptographic hash functions like SHA-256 to produce outputs that meet certain criteria, such as starting with a number of zero bits.

- In blockchain mining, miners seek a nonce that causes the hash of a block to have a specific number of leading zeros, which proves computational effort.

- This task demonstrates the same core idea by searching for inputs (words) whose hashes naturally meet that condition.

- The rarity of leading zeros showcases the pseudo-random nature of SHA-256 and the infeasibility of predicting or preselecting favorable inputs.

### **Step 1: Load Dictionary and Initialize Tracking Variables**

- **Definition**

    - Loads all words from `dictionary.txt` into a lowercase set to eliminate duplicates.

    - Initializes `max_leading_zeros` and `best_words` to track the best result encountered during processing.
    
    - Confirms the total number of words loaded.

In [206]:
with open("dictionary.txt", "r") as file:
    english_words = set(word.strip().lower() for word in file if word.strip())

max_leading_zeros = 0
best_words = []

print(f"Loaded 'dictionary.txt' with {len(english_words)} words.")

Loaded 'dictionary.txt' with 233614 words.


#### **Test Class: `TestSetup`**

- **Test Approach**
    - Validates that the dictionary file exists, is not empty, and is correctly processed into a usable set.

    - Checks initialization of key tracking variables.

- **Verification**
    - Ensures `dictionary.txt` is found and contains non-empty lines.

    - Confirms `english_words` is a populated set of lowercase strings.

    - Verifies expected words like `'apple'` are present.
    
    - Confirms that `max_leading_zeros` starts at 0 and `best_words` starts empty.

- **Outcome**
    - All checks pass, confirming proper file loading, data structure integrity, and correct initial states.

In [207]:
class TestSetup(unittest.TestCase):

    def test_dictionary_file_exists_and_not_empty(self):
        self.assertTrue(os.path.exists("dictionary.txt"), "dictionary.txt should exist.")
        with open("dictionary.txt", "r") as f:
            lines = [line.strip() for line in f if line.strip()]
        self.assertGreater(len(lines), 0, "dictionary.txt should not be empty.")
        print("Test passed: dictionary.txt exists and contains words.")

    def test_english_words_set(self):
        self.assertIsInstance(english_words, set, "english_words should be a set.")
        self.assertGreater(len(english_words), 0, "english_words should not be empty.")
        sample = next(iter(english_words))
        self.assertIsInstance(sample, str, "Each word should be a string.")
        print("Test passed: english_words is a non-empty set of strings.")

    def test_common_word_exists(self):
        self.assertIn('apple', english_words, "'apple' should be in the English word list.")
        print("Test passed: Common word 'apple' found in english_words.")

    def test_max_leading_zeros_initial(self):
        self.assertEqual(max_leading_zeros, 0, "max_leading_zeros should start at 0.")
        print("Test passed: max_leading_zeros is initialized to 0.")

    def test_best_words_initial(self):
        self.assertEqual(best_words, [], "best_words should start as an empty list.")
        print("Test passed: best_words is initialized as an empty list.")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestSetup))

test_best_words_initial (__main__.TestSetup.test_best_words_initial) ... ok
test_common_word_exists (__main__.TestSetup.test_common_word_exists) ... ok
test_dictionary_file_exists_and_not_empty (__main__.TestSetup.test_dictionary_file_exists_and_not_empty) ... 

Test passed: best_words is initialized as an empty list.
Test passed: Common word 'apple' found in english_words.
Test passed: dictionary.txt exists and contains words.


ok
test_english_words_set (__main__.TestSetup.test_english_words_set) ... ok
test_max_leading_zeros_initial (__main__.TestSetup.test_max_leading_zeros_initial) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.162s

OK


Test passed: english_words is a non-empty set of strings.
Test passed: max_leading_zeros is initialized to 0.


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### **Step 2: Count Leading Zero Bits in SHA-256 Hashes**

- **Definition**

    - Defines a helper function `count_leading_zero_bits()` to count the number of leading `'0'`s in a binary string.

    - Converts the SHA-256 hex digest to a 256-bit binary string.
    
    - Processes each word in the dictionary:

        - Hashes the word

        - Counts the leading zeros
        
        - Updates tracking variables if a new maximum is found or tied

In [208]:
# Function to count leading zero bits in a binary string
def count_leading_zero_bits(binary_string):
    return len(binary_string) - len(binary_string.lstrip('0'))

# Process each word
for word in english_words:
    word = word.lower()
    h = hashlib.sha256(word.encode()).hexdigest() 
    b = bin(int(h, 16))[2:].zfill(256)  
    count = count_leading_zero_bits(b)

    if count > max_leading_zeros:
        max_leading_zeros = count
        best_words = [word]
    elif count == max_leading_zeros:
        best_words.append(word)

#### **Test Class: `TestSHA256BitAnalysis`**

- **Test Approach**
    - Verifies the correctness and consistency of hash processing and bit-counting logic.

- **Verification**
    - Validates that `count_leading_zero_bits()` returns the correct count for test inputs.
    - Confirms that SHA-256 binary strings are 256 bits and contain only binary characters.
    - Ensures that hashing the same word yields consistent results.
    - Tests the logic that tracks which words have the most leading zeros.

- **Outcome**
    - All functional and edge-case tests pass.
    - Confirms SHA-256 conversions and tracking logic behave as expected.

In [209]:
class TestSHA256BitAnalysis(unittest.TestCase):

    def test_leading_zero_bits_count(self):
        self.assertEqual(count_leading_zero_bits('00001111'), 4)
        self.assertEqual(count_leading_zero_bits('11111111'), 0)
        self.assertEqual(count_leading_zero_bits('00000000'), 8)
        print("Test passed: count_leading_zero_bits returns correct values.")

    def test_sha256_binary_conversion(self):
        word = 'example'
        h = hashlib.sha256(word.encode()).hexdigest()
        b = bin(int(h, 16))[2:].zfill(256)
        self.assertEqual(len(b), 256)
        self.assertTrue(all(c in '01' for c in b))
        print("Test passed: SHA-256 binary conversion produces 256-bit binary string.")

    def test_sha256_consistency(self):
        word = 'blockchain'
        h1 = hashlib.sha256(word.encode()).hexdigest()
        h2 = hashlib.sha256(word.encode()).hexdigest()
        self.assertEqual(h1, h2)
        print("Test passed: SHA-256 is deterministic for same input.")

    def test_max_leading_zero_tracking(self):
        test_words = ['aaaaa', 'zzzzz', 'hello']
        max_zeros = 0
        best = []
        for w in test_words:
            h = hashlib.sha256(w.encode()).hexdigest()
            b = bin(int(h, 16))[2:].zfill(256)
            count = count_leading_zero_bits(b)
            if count > max_zeros:
                max_zeros = count
                best = [w]
            elif count == max_zeros:
                best.append(w)
        self.assertIn(best[0], test_words)
        self.assertGreaterEqual(max_zeros, 0)
        print("Test passed: leading-zero tracking logic works correctly.")

unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestSHA256BitAnalysis))


test_leading_zero_bits_count (__main__.TestSHA256BitAnalysis.test_leading_zero_bits_count) ... ok
test_max_leading_zero_tracking (__main__.TestSHA256BitAnalysis.test_max_leading_zero_tracking) ... ok
test_sha256_binary_conversion (__main__.TestSHA256BitAnalysis.test_sha256_binary_conversion) ... ok
test_sha256_consistency (__main__.TestSHA256BitAnalysis.test_sha256_consistency) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.010s

OK


Test passed: count_leading_zero_bits returns correct values.
Test passed: leading-zero tracking logic works correctly.
Test passed: SHA-256 binary conversion produces 256-bit binary string.
Test passed: SHA-256 is deterministic for same input.


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

#### **Step 3: Output Maximum Result**

- **Definition**

    - After evaluating all dictionary words, prints:

        - The maximum number of leading zero bits observed

        - The list of words that produced that number
        
    - Demonstrates a real-world analog of proof-of-work discovery, using natural language instead of a nonce.

In [216]:
# Show results
print(f"Maximum leading zero bits: {max_leading_zeros}")
print("Top word(s) with the most leading zero bits:")

# Sort alphabeticall
for w in sorted(best_words):
    print(f"\n  {w}")

Maximum leading zero bits: 16
Top word(s) with the most leading zero bits:

  guilefulness

  mismatchment


### **Conclusion: Hash-Based Proof of Work Simulation**

- This task successfully demonstrated a simplified proof-of-work search over natural language input.

- The SHA-256 hash of each word was converted to binary and analyzed for leading zero bits.

- Tracking logic was implemented to find the best results, verified by unit tests.

- The simulation provides insight into how computational difficulty and randomness emerge in hash-based systems.

- This concept is directly applicable to blockchain mining, where miners must search through inputs to find rare hash outputs.

## **Task 7 - Turing Machines**

**Purpose**

- This task simulates a basic Turing Machine that performs binary increment.

- The machine reads a binary string (the tape), moves its head to the rightmost bit, and simulates binary addition with carry handling.

- The function models Turing-style head movement and state changes without using loops or arithmetic shortcuts.

- The goal is to explore computation as a sequence of discrete, rule-based steps, as in theoretical models of computation.

**Theoretical Context**

- Turing Machines are a foundational model in computational theory, used to define the limits of what can be computed.

- They consist of a tape (infinite in theory), a read/write head, and a set of deterministic transition rules.

- This simulation models a single transition process: incrementing a binary number by 1 using tape manipulation and head movement.

- It demonstrates the universality of the Turing Machine model—even simple arithmetic can be performed through symbol rewriting alone.

- This task connects to topics like computability, automata theory, and the Church-Turing thesis.

### **Step 1: Simulate Binary Increment on a Turing Machine**

- **Definition**

    - Converts the input tape (string) into a mutable list of symbols.

    - Moves the head to the rightmost bit of the tape.

    - Applies binary addition logic with carry:

        - If the current bit is `0`, it’s changed to `1` and the process halts.

        - If the bit is `1`, it’s set to `0`, and the head moves left to propagate the carry.

        - If all bits are `1`, a new `1` is added to the front of the tape.
        
    - Returns the new tape as a string representing the incremented binary number.

In [160]:
def turing_add_one(tape):
    tape = list(tape)
    head = 0

    # Move to the end of the tape (rightmost bit)
    while head < len(tape):
        head += 1
    head -= 1

    # Perform binary addition (carry = 1)
    while head >= 0:
        if tape[head] == '0':
            tape[head] = '1'
            break
        elif tape[head] == '1':
            tape[head] = '0'
            head -= 1
        else:
            raise ValueError("Invalid symbol on tape")

    # If the carry propagates to the leftmost bit, add a new bit
    if head < 0:
        tape = ['1'] + tape

    return ''.join(tape)

#### **Test Class: `TestTuringAddOne`**

- **Test Approach**
    - Validates the Turing Machine binary increment function against multiple known inputs and outputs.

    - Includes edge cases like all-ones (`'111'`), leading zeros, and simple 1-bit cases.

    - Prints confirmation of each test's success.

- **Verification**
    - Checks each output string matches the correct binary result after increment.

    - Ensures no input formats cause the machine to fail or misbehave.

    - Confirms carry propagation works as expected across the full tape.

- **Outcome**
    - All tests pass.
    
    - Function correctly simulates a Turing Machine increment operation for all test cases provided.

In [ ]:
class TestTuringAddOne(unittest.TestCase):

    def test_turing_add_one(self):
        print("\nStarting validation for Turing Machine binary increment function...\n")

        test_cases = {
            '0':      '1',
            '1':      '10',
            '1010':   '1011',
            '111':    '1000',
            '100111': '101000',
            '000':    '001',
            '001':    '010'
        }

        # Test each case
        for binary_input, expected_output in test_cases.items():
            result = turing_add_one(binary_input)
            self.assertEqual(result, expected_output)
            print(f"Test passed: Input '{binary_input}' -> Output '{result}'")

        print("\nAll Turing Machine increment tests completed successfully.")

# Run the tests
unittest.TextTestRunner(verbosity=2).run(unittest.TestLoader().loadTestsFromTestCase(TestTuringAddOne))

test_turing_add_one (__main__.TestTuringAddOne.test_turing_add_one) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



Starting validation for Turing Machine binary increment function...

Test passed: Input '0' -> Output '1'
Test passed: Input '1' -> Output '10'
Test passed: Input '1010' -> Output '1011'
Test passed: Input '111' -> Output '1000'
Test passed: Input '100111' -> Output '101000'
Test passed: Input '000' -> Output '001'
Test passed: Input '001' -> Output '010'

All Turing Machine increment tests completed successfully.


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### **Step 2: Demonstrate Binary Increment**

- **Definition**

    - Demonstrates a working example using the tape `'100111'`.

    - Calls the increment function and prints both the input and output tapes.
    
    - Confirms the machine correctly produces the incremented result `'101000'`.

In [162]:
initial = "100111"
result = turing_add_one(initial)
print("Input Tape: ", initial)
print("Output Tape:", result)

Input Tape:  100111
Output Tape: 101000


### **Conclusion: Simulated Turing Machine for Binary Increment**

- This task successfully demonstrated the simulation of a simple Turing Machine that increments a binary number.

- Unit tests validated the correctness of the function across various inputs, including those with leading zeros and different bit lengths.

- This task highlights how fundamental computational logic can be implemented using rule-based, tape-driven systems, consistent with the Turing model of universal computation.


## **Task 8 - Computational Complexity**

**Purpose**

- This task analyzes the comparison count performance of the bubble sort algorithm under all permutations of a fixed-size input.

- Two implementations are used: a basic bubble sort and an optimized version with early exit on sorted input.

- The number of comparisons made during sorting is counted for each permutation.

- Output includes first and last 10 permutations to summarize results without excessive verbosity.

**Theoretical Context**

- Bubble sort is a simple comparison-based sorting algorithm with a worst-case time complexity of O(n²).

- Measuring the number of comparisons across permutations helps visualize best-case, average-case, and worst-case behavior.

- The optimized version improves best-case performance (O(n)) when the input is already sorted, by checking for swaps.

- This task demonstrates how the performance of bubble sort varies across all 120 permutations of a 5-element list (n = 5), allowing practical observation of its algorithmic complexity.

### **Step 1: Implement Basic Bubble Sort with Comparison Counting**

- **Definition**

    - Implements standard bubble sort with an added counter for the number of comparisons.

    - Performs nested passes through the list, comparing adjacent elements and swapping if needed.
    
    - Returns the sorted array and total comparisons made during sorting.

In [226]:
def bubble_sort_with_comparisons(arr):
    arr = list(arr)
    n = len(arr)
    comparisons = 0
    for i in range(n):
        for j in range(n - i - 1):
            comparisons += 1
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr, comparisons

# Generate results
L = [1, 2, 3, 4, 5]
perms = list(itertools.permutations(L))
results = [(p, bubble_sort_with_comparisons(p)) for p in perms]

### **Step 2: Evaluate Bubble Sort on All Permutations**

- **Definition**

    - Generates all 120 permutations of the list `[1, 2, 3, 4, 5]`.
    
    - Applies `bubble_sort_with_comparisons()` to each permutation.

    - Flattens and formats the results into `(input, comparisons, output)` triplets.
    
    - Prints the first 10 and last 10 results with a separator for readability.

In [219]:
# Format output as (perm, comparisons, sorted)
def format_result(p, s, c):
    return f"Input: {p} -> Comparisons: {c}, Sorted: {s}"

# Extract and flatten results
flattened_results = [(p, s, c) for p, (s, c) in results]

# First 10
print("Bubble Sort Comparison Results (first 10):\n")
for p, s, c in flattened_results[:10]:
    print(format_result(p, s, c))

# Ellipsis separator
print("\n...\n")

# Last 10
print("Bubble Sort Comparison Results (last 10):\n")
for p, s, c in flattened_results[-10:]:
    print(format_result(p, s, c))

Bubble Sort Comparison Results (first 10):

Input: (1, 2, 3, 4, 5) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 3, 5, 4) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 4, 3, 5) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 4, 5, 3) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 5, 3, 4) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 5, 4, 3) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (1, 3, 2, 4, 5) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (1, 3, 2, 5, 4) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (1, 3, 4, 2, 5) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (1, 3, 4, 5, 2) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]

...

Bubble Sort Comparison Results (last 10):

Input: (5, 3, 2, 1, 4) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (5, 3, 2, 4, 1) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (5, 3, 4, 1, 2) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (5, 3, 4, 2, 1) -> Comparison

### **Step 3: Implement Optimised Bubble Sort with Early Termination**

- **Definition**

    - Enhanced version of bubble sort that introduces an early-exit condition for improved efficiency on already sorted input.

    - At the beginning of each outer loop pass, a `swapped` flag is set to `False`.

    - As elements are compared and potentially swapped, this flag is set to `True` if any swap occurs.

    - If an entire pass completes with no swaps, the list is already sorted, and the algorithm exits early—saving unnecessary comparisons.

    - This optimised version reduces best-case time complexity from O(n²) to O(n), while retaining the same worst-case performance.
    
    - Still tracks the number of comparisons to allow a fair performance comparison with the basic bubble sort implementation.


In [ ]:
# Bubble sort with comparison counting
def bubble_sort_optimised(arr):
    arr = list(arr)
    n = len(arr)
    comparisons = 0
    for i in range(n):
        swapped = False
        for j in range(n - i - 1):
            comparisons += 1
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
        if not swapped:
            break
    return arr, comparisons

# Generate permutations and results
L = [1, 2, 3, 4, 5]
perms = list(itertools.permutations(L))
results = [(str(p), c, str(s)) for p, (s, c) in zip(perms, map(bubble_sort_optimised, perms))]

### **Step 4: Evaluate Optimised Bubble Sort on All Permutations**

- **Definition**

    - Repeats the same process as Step 2, but using `bubble_sort_optimised()` instead.

    - Collects the number of comparisons and final sorted results for each permutation.

    - Displays first 10 and last 10 results for simplicity and readability.

In [222]:
# Format and print result
def format_result(p, c, s):
    return f"Input: {p} -> Comparisons: {c}, Sorted: {s}"

# First 10 results
print("Optimised Bubble Sort Comparison Results (first 10):\n")
for p, c, s in results[:10]:
    print(format_result(p, c, s))

# Ellipsis separator
print("\n...\n")

# Last 10 results
print("Optimised Bubble Sort Comparison Results (last 10):\n")
for p, c, s in results[-10:]:
    print(format_result(p, c, s))


Optimised Bubble Sort Comparison Results (first 10):

Input: (1, 2, 3, 4, 5) -> Comparisons: 4, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 3, 5, 4) -> Comparisons: 7, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 4, 3, 5) -> Comparisons: 7, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 4, 5, 3) -> Comparisons: 9, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 5, 3, 4) -> Comparisons: 7, Sorted: [1, 2, 3, 4, 5]
Input: (1, 2, 5, 4, 3) -> Comparisons: 9, Sorted: [1, 2, 3, 4, 5]
Input: (1, 3, 2, 4, 5) -> Comparisons: 7, Sorted: [1, 2, 3, 4, 5]
Input: (1, 3, 2, 5, 4) -> Comparisons: 7, Sorted: [1, 2, 3, 4, 5]
Input: (1, 3, 4, 2, 5) -> Comparisons: 9, Sorted: [1, 2, 3, 4, 5]
Input: (1, 3, 4, 5, 2) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]

...

Optimised Bubble Sort Comparison Results (last 10):

Input: (5, 3, 2, 1, 4) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (5, 3, 2, 4, 1) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (5, 3, 4, 1, 2) -> Comparisons: 10, Sorted: [1, 2, 3, 4, 5]
Input: (5, 3, 4, 2, 1) ->

### **Conclusion: Bubble Sort Comparison Analysis**

- This task explored the computational complexity of bubble sort across all permutations of a 5-element list.

- The standard and optimized implementations were analyzed in terms of comparison count for each permutation.

- As expected, the optimized version reduced comparisons significantly for already sorted inputs.

- The full permutation analysis helped visualize how performance varies from best to worst case.

- This task reinforces understanding of practical algorithmic behavior, even for small input sizes.